# Manage Quota

Note: 
- Geo Engine has to have "pro" feature enabled
- quota_check has to be enabled in Settings

In [3]:
import geoengine as ge

from datetime import datetime

# Select a user

In [4]:
email = "foo@example.com"
password = "secret12345"

# register the user, if it doesn't exist yet
import requests as re
re.post("http://localhost:3030/api/user", json={"email": email, "password": password, "realName": "Foo Bar"})

<Response [200]>

## Initialize as user

In [5]:
user_client = ge.create_client("http://localhost:3030/api", credentials=(email, password))

user_id = user_client.get_session().user_id

## Access own quota

In [6]:
user_client.quota_get()

{'available': 1, 'used': 0}

## Try to run a query (fails, because quota is exhausted)

In [7]:
ports = user_client.workflow_register({
    "type": "Vector",
    "operator": {
        "type": "OgrSource",
        "params": {
            "data": "ne_10m_ports",
            "attributeProjection": None
        }
    }
})

time = datetime.strptime(
    '2014-04-01T12:00:00.000Z', "%Y-%m-%dT%H:%M:%S.%f%z")

try:
    df = user_client.workflow_as_dataframe(
        ports,
        ge.QueryRectangle(
            ge.BoundingBox2D(-180.0, -90.0, 180.0, 90.0),
            ge.TimeInterval(time, time),
            ge.SpatialResolution(0.1, 0.1)
            )
    )
except ge.GeoEngineException as e:
    print(e)

## Initialize Geo Engine as Admin

In [8]:
admin_client = ge.create_client("http://localhost:3030/api", ("admin@localhost", "adminadmin"))

## Access user quota

In [9]:
admin_client.quota_get(user_id)

{'available': 0, 'used': 1}

## Update user quota

In [10]:
admin_client.quota_update(user_id, 1000)

## Verify quota update worked

In [11]:
admin_client.quota_get(user_id)

{'available': 1000, 'used': 1}

# Go back to the regular user

In [12]:
user_client.quota_get()

{'available': 1000, 'used': 1}

# Rerun the workflow, works now

In [13]:

df = user_client.workflow_as_dataframe(
        ports,
        ge.QueryRectangle(
            ge.BoundingBox2D(-180.0, -90.0, 180.0, 90.0),
            ge.TimeInterval(time, time),
            ge.SpatialResolution(0.1, 0.1)
            )
    )

df.head()

,geometry,featurecla,name,natlscale,scalerank,website,start,end
0,POINT (-69.92356 12.43750),Port,Sint Nicolaas,5.0,8,www.rocargo.com/SanNicolas.html,NaT,NaT
1,POINT (-58.95141 -34.15333),Port,Campana,5.0,8,www.consejoportuario.com.ar,NaT,NaT
2,POINT (-59.00495 -34.09889),Port,Zarate,5.0,8,www.consejoportuario.com.ar,NaT,NaT
3,POINT (-62.10088 -38.89444),Port,Puerto Belgrano/Bahia Blanca,5.0,8,None,NaT,NaT
4,POINT (-62.30053 -38.78306),Port,Puerto Galvan/Bahia Blanca,5.0,8,None,NaT,NaT


In [14]:
import time
time.sleep(3) # Sleep for 3 seconds

## Verify that the used quota was recorded

In [15]:

user_client.quota_get()

{'available': 999, 'used': 2}